In [ ]:
#import relevant libraries
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import selenium.webdriver as webdriver
from selenium import webdriver
from bs4 import BeautifulSoup 
from pyparsing import line
import time
import csv


# intialize chrome service
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36"
driver_path = r"C:\chromedriver.exe"
chrome_service = Service(driver_path)
chrome_options = Options()
chrome_options.add_argument(f'user-agent={user_agent}')
browser = webdriver.Chrome(service=chrome_service, options=chrome_options)


#read the login credentials
login =open('login_credential.txt')
line = login.readlines()
email = line[0]
pwd = line[1]
searchQuery =line[2]
input_pages = int(line[3])

# navigate to a website
browser.get('https://www.linkedin.com/login')
time.sleep(3)

#input the email into the website
email_field = browser.find_element(by=By.ID, value ="username")
email_field.send_keys(email)
time.sleep(3)

#input the password into the website
password_field = browser.find_element(by=By.ID, value ='password')
password_field.send_keys(pwd)
time.sleep(5)

#click the send button
#loginbutton = browser.find_element(by=By.CLASS_NAME, value="btn__primary--large from__button--floating")
#loginbutton.click()
#time.sleep(10)

#input the search query
searchbutton = browser.find_element(by=By.XPATH, value ='//*[@id="global-nav-typeahead"]/input')
searchbutton.send_keys(searchQuery)
time.sleep(3)

#search
searchbutton.send_keys(Keys.RETURN)
time.sleep(3)

#PeopleResult = browser.find_element(by=By.XPATH, value ='//*[@id="search-reusables__filters-bar"]/ul/li[2]/button')
#PeopleResult.click()
#time.sleep(3)


#function to extract profile url link from the different profile displayed
def GetURL():
    all_profile_URL=[]
    pagesource = BeautifulSoup(browser.page_source)
    profiles = pagesource.find_all('a', class_ = 'app-aware-link')
    for profile in profiles:
        profile_URL = profile.get('href')
        if 'https://www.linkedin.com/in' in profile_URL and  profile_URL not in all_profile_URL:
            all_profile_URL.append(profile_URL)
    return all_profile_URL
    
#loop through the number of pages to get the url for the different profile
def GetURLonPage():
    UrLs_all_page = []
    for page in range(input_pages):
        URLs_one_page = GetURL()
        time.sleep(3)
        browser.execute_script('window.scrollTo(0,document.body.scrollHeight);')
        time.sleep(2)
        nextbutton = browser.find_element(by=By.CLASS_NAME, value="artdeco-pagination__button--next")
        nextbutton.click()
        UrLs_all_page = UrLs_all_page + URLs_one_page
        time.sleep(3)
    return UrLs_all_page

Urls_all_page = GetURLonPage()
#create a csv file to store the relevant information
with open('output.csv', 'w', newline='') as file_output:
    headers = ['Name','Title','Location','URL']
    writer = csv.DictWriter(file_output, delimiter=',' ,lineterminator="\n", fieldnames=headers)
    writer.writeheader()
#loop the profile link to extract the relevant information
    for linkedin_URL in Urls_all_page:
        browser.get(linkedin_URL)
        time.sleep(2)
        page_source = BeautifulSoup(browser.page_source, 'html.parser')
        info_div = page_source.find('div', class_="mt2 relative")
        name = info_div.find("h1", class_ ='text-heading-xlarge inline t-24 v-align-middle break-words').get_text().strip()
        title_name = info_div.find("div", class_ ='text-body-medium break-words').get_text().strip()
        city_name = info_div.find("span", class_="text-body-small inline t-black--light break-words").get_text().strip()
        writer.writerow({headers[0]:name, headers[1]:title_name, headers[2]:city_name,headers[3]:linkedin_URL})

